In [2]:
!pip install seaborn

     ---------------------------------------- 0.0/162.3 kB ? eta -:--:--
     -- ------------------------------------- 10.2/162.3 kB ? eta -:--:--
     --------- --------------------------- 41.0/162.3 kB 487.6 kB/s eta 0:00:01
     ---------------------------- --------- 122.9/162.3 kB 1.0 MB/s eta 0:00:01
     -------------------------------------- 162.3/162.3 kB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/294.9 kB ? eta -:--:--
   ------------------- -------------------- 143.4/294.9 kB 4.3 MB/s eta 0:00:01
   ---------------------------------- ----- 256.0/294.9 kB 4.0 MB/s eta 0:00:01
   ---------------------------------------- 294.9/294.9 kB 2.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/7.5 MB ? eta -:--:--
   - -------------------------------------- 0.3/7.5 MB 8.9 MB/s eta 0:00:01
   -- ------------------------------------- 0.5/7.5 MB 5.4 MB/s eta 0:00:02
   --- ------------------------------------ 0.7/7.5 MB 6.4 MB/s eta 0:00:02
   --

In [5]:
!pip install scikit-learn

     ---------------------------------------- 0.0/59.0 kB ? eta -:--:--
     ------ --------------------------------- 10.2/59.0 kB ? eta -:--:--
     -------------------------- ----------- 41.0/59.0 kB 495.5 kB/s eta 0:00:01
     -------------------------------------- 59.0/59.0 kB 524.0 kB/s eta 0:00:00
   ---------------------------------------- 0.0/9.3 MB ? eta -:--:--
   ---------------------------------------- 0.1/9.3 MB 3.2 MB/s eta 0:00:03
    --------------------------------------- 0.2/9.3 MB 2.1 MB/s eta 0:00:05
   - -------------------------------------- 0.3/9.3 MB 2.0 MB/s eta 0:00:05
   -- ------------------------------------- 0.5/9.3 MB 2.8 MB/s eta 0:00:04
   --- ------------------------------------ 0.9/9.3 MB 3.9 MB/s eta 0:00:03
   ---- ----------------------------------- 1.0/9.3 MB 4.1 MB/s eta 0:00:03
   ------ --------------------------------- 1.5/9.3 MB 4.7 MB/s eta 0:00:02
   ------- -------------------------------- 1.7/9.3 MB 4.8 MB/s eta 0:00:02
   --------- -----

In [15]:
import pandas as pd
import seaborn as sns
import requests
from sklearn.ensemble import IsolationForest

In [7]:
df = pd.read_csv("keylogs.csv")

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Keystroke Latency   7 non-null      float64
 1   Backspace Count     7 non-null      int64  
 2   Right Click Count   7 non-null      int64  
 3   Left Click Count    7 non-null      int64  
 4   Malware Word Count  7 non-null      int64  
dtypes: float64(1), int64(4)
memory usage: 408.0 bytes


In [9]:
anomaly_inputs = ['Keystroke Latency','Backspace Count','Right Click Count','Left Click Count','Malware Word Count']

In [10]:
model_IF = IsolationForest(contamination=0.1, random_state=42)

In [11]:
model_IF.fit(df[anomaly_inputs])

IsolationForest(contamination=0.1, random_state=42)

In [12]:
df['anomaly_scores'] = model_IF.decision_function(df[anomaly_inputs])

In [13]:
df['anomaly'] = model_IF.predict(df[anomaly_inputs])

In [14]:
df.loc[:, ['Keystroke Latency','Backspace Count','Right Click Count','Left Click Count','Malware Word Count','anomaly_scores','anomaly'] ]

,Keystroke Latency,Backspace Count,Right Click Count,Left Click Count,Malware Word Count,anomaly_scores,anomaly
0,0.000,3,0,6,0,0.053228,1
1,145.944,3,0,4,0,0.209528,1
2,160.555,3,0,9,0,0.148474,1
3,145.085,4,0,4,0,0.176045,1
4,100.085,7,5,22,0,-0.079843,-1
5,131.258,0,0,0,0,0.101271,1
6,166.521,0,0,3,0,0.133838,1


In [16]:
url = 'http://localhost:8080/api/v1/data/alerts'

In [18]:
for index, row in df.iterrows():
    if row['anomaly'] == -1:
        response = requests.post(url, data='An intruder has been found from keylogger information')
        print(response.text)

Alert received
